In [6]:
!pip install lightgbm

ERROR: Could not install packages due to an OSError: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\ProgramData\\Anaconda3\\Lib\\site-packages\\lightgbm\\lib_lightgbm.dll'
Consider using the `--user` option or check the permissions.



In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [ ]:
tongue = pd.read_csv("tongue.csv")    #reading in the dataset

In [ ]:
tongue.head()     #displating the first five rows of the dataset

In [ ]:
#renaming the column for readablity pupose
tongue = tongue.rename(columns = {"op": "operation_status", "rt": "radiation_therapy", "ct": "chemotherapy_status"})

In [ ]:
tongue.head() #displaying the first five rows of the dataset

In [ ]:
tongue.shape  #displaying the number of rows and columns of the dataset

In [ ]:
tongue.isna().sum()    #checking for null rows